In [1]:
import pytorch_lightning as pl
import pandas as pd
import os

from src.misc.extracting_runs import extract_runs
import src.utils as utils
from src.data.dataloader import mtlDataModule
from src.pipeline.lightning_frame import mtlMayhemModule

WORKDIR_HPC = "/work3/s202821/models/"
OUTDIR_HPC = "/work3/s202821/emp_models/"

WORKDIR_LOCAL = "/home/bnbj/repos/multitask-mayhem/"

os.chdir(WORKDIR_LOCAL)

/home/bnbj/miniconda3/envs/multitask-mayhem/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
os.getcwd()

'/home/bnbj/repos/multitask-mayhem'

## Separate runs on HPC

In [2]:
# extract_runs(workdir = WORKDIR_HPC, outdir = OUTDIR_HPC)

## Run model on test set

In [3]:
runs_df = pd.read_csv("models/emp_models/runs_list.csv", index_col=0)
runs_list = runs_df["name"].values.tolist()
runs_df.head()

,name
0,frcnn-hybrid_10v1_480x640_23-02-22T123517
1,frcnn-hybrid_480x640_23-02-22T115457
2,frcnn-hybrid_2v1_23-02-21T182529
3,frcnn-hybrid_dwa-10v1-cagrad_23-02-21T182427
4,frcnn-hybrid_dwa-10v1-graddrop_23-02-21T182345


In [4]:
run_name = runs_list[1]
run_name

'frcnn-hybrid_480x640_23-02-22T115457'

In [5]:
config_path = f"models/emp_models/{run_name}/{run_name}.yaml"
temp_path = f"models/emp_models/{run_name}/temp_{run_name}.yaml"

config = utils.load_yaml(config_path)

utils.change_paths_in_config(
    input_yaml = config_path,
    out_yaml = config_path,
    data_root = "data/interim",
    test_manifest = "data/test/manifest.json",
    model_out_path = "models/emp_models"
)




In [6]:
# initialize data pipeline
lightning_datamodule = mtlDataModule(config_path)

# initialize training pipeline
lightning_module = mtlMayhemModule(config_path)

100%|██████████| 8/8 [00:04<00:00,  1.72it/s]


In [7]:
# TRAINER #
trainer = pl.Trainer(
    accelerator="auto",
    devices=1,
    enable_checkpointing=not (lightning_datamodule.config["debug"]),
    default_root_dir=lightning_module.path_dict["checkpoints_path"],
)

# start training
trainer.test(model=lightning_module, datamodule=lightning_datamodule)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: models/emp_models/frcnn-hybrid_480x640_23-02-22T115457/checkpoints/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/bnbj/miniconda3/envs/multitask-mayhem/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing DataLoader 0: 100%|██████████| 260/260 [00:11<00:00, 22.68it/s]


[{}]